## Prepare data

In [62]:
import torch
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_device
import numpy as np

In [63]:
#!pip install evaluate datasets transformers accelerate==1.9.0

In [64]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

df = pd.read_csv("data/acc_data.csv")
df.head()

,recordId,gpt4o_judge_score,nova_judge_score,llama3_judge_score,majority_value,agreement_percentage,writing_id,task_id,level_title,activity_instructions,student_submission
0,CALL0000322,3.0,4.0,3.0,3.0,66.666667,08e77bf2-2a0d-4098-b5ec-40f593ca6336,de7a1159-0c06-49bf-a273-7bdd195d6a32,11-Upper Intermediate,You are applying for a position as a manager a...,I am writing to apply for the manager position...
1,CALL0002886,0.0,0.0,0.0,0.0,100.000000,f6a2b5e6-ceb8-4575-ac74-ada1ab9f8cc7,7ae47a96-722c-4b2b-ad07-3716f2aef85f,8-Intermediate,You're going to write an email about some futu...,You're going to write an email about some futu...
2,CALL0001805,5.0,5.0,5.0,5.0,100.000000,f8823fe1-ca4b-43dc-9685-85009a105efa,cfb893f1-bee1-4894-99f3-43af284493a5,3-Beginner,Write a paragraph about a job your friend has ...,My friend João worked at TechNova Soluções fro...
3,CALL0000693,5.0,4.0,5.0,5.0,66.666667,2ddd84ba-aebc-4808-b270-ca3bc19cb15a,03849587-d03b-45ed-8534-c34536867a8a,12-Upper Intermediate,Write a letter to apologize to someone for som...,"Dear Alex,\n\nI want to sincerely apologize fo..."
4,CALL0000291,5.0,5.0,5.0,5.0,100.000000,57ac94ab-2084-4a23-a6bf-61d71cd1900a,23c3d59a-61d5-470f-90d1-1217b0006929,10-Upper Intermediate,"An old friend has emailed you, describing what...",Subject: So Good to Hear From You! \n\nStan! ...


In [65]:
df["task_id"].value_counts()

task_id
03849587-d03b-45ed-8534-c34536867a8a    99
43d154ea-bd72-4b54-9339-7c12f992b56a    99
8f1a0c16-d259-4737-af06-b6b6eb9e9799    98
bd802cec-aaac-4a24-a654-01d84a6a972d    97
f03e111e-8a30-48fe-991b-6adfea2444ad    97
9032f3ed-5da1-4efe-b4dd-c9470bb35d65    97
6e8793e1-760b-43e0-a8a9-44dc11336782    96
b69c3bc3-65b0-4518-8a34-1d12e67299ec    96
f425c7a4-1713-4a57-b978-313423486bbd    96
28757296-8797-428f-a562-0b03c304b341    94
d040caa5-7bd8-412c-aee2-3d9010b91821    94
d8e67c00-1b68-450a-93f8-ed8f40a1c167    94
b3285147-d9b6-42f7-9ffb-7482cd450db7    93
dcdc664b-404c-407a-99a6-b9a1f99a859a    93
b4ba34ff-949d-4a82-9f56-ea472f177a71    92
5c44f26d-2f5c-4039-bcf5-378754d2fe4e    91
de7a1159-0c06-49bf-a273-7bdd195d6a32    91
7ae47a96-722c-4b2b-ad07-3716f2aef85f    91
c7972fb8-c4a3-4fd8-8892-33804a60162e    90
48b78e21-bcae-4548-a0e9-1e150343125a    89
f7b59a4c-48b9-4c6b-a670-3c98806ec805    89
f7a3382d-3200-40de-98ed-11b7fe56a550    89
5949887a-c75f-46ad-95d6-9775e932a1e6    89
fa1

In [66]:
df['text'] = (
    "Prompt Level: " + df['level_title'].astype(str) +
    " [SEP] Prompt: " + df['activity_instructions'] +
    " [SEP] Response: " + df['student_submission']
)


In [67]:
df.head()

,recordId,gpt4o_judge_score,nova_judge_score,llama3_judge_score,majority_value,agreement_percentage,writing_id,task_id,level_title,activity_instructions,student_submission,text
0,CALL0000322,3.0,4.0,3.0,3.0,66.666667,08e77bf2-2a0d-4098-b5ec-40f593ca6336,de7a1159-0c06-49bf-a273-7bdd195d6a32,11-Upper Intermediate,You are applying for a position as a manager a...,I am writing to apply for the manager position...,Prompt Level: 11-Upper Intermediate [SEP] Prom...
1,CALL0002886,0.0,0.0,0.0,0.0,100.000000,f6a2b5e6-ceb8-4575-ac74-ada1ab9f8cc7,7ae47a96-722c-4b2b-ad07-3716f2aef85f,8-Intermediate,You're going to write an email about some futu...,You're going to write an email about some futu...,Prompt Level: 8-Intermediate [SEP] Prompt: You...
2,CALL0001805,5.0,5.0,5.0,5.0,100.000000,f8823fe1-ca4b-43dc-9685-85009a105efa,cfb893f1-bee1-4894-99f3-43af284493a5,3-Beginner,Write a paragraph about a job your friend has ...,My friend João worked at TechNova Soluções fro...,Prompt Level: 3-Beginner [SEP] Prompt: Write a...
3,CALL0000693,5.0,4.0,5.0,5.0,66.666667,2ddd84ba-aebc-4808-b270-ca3bc19cb15a,03849587-d03b-45ed-8534-c34536867a8a,12-Upper Intermediate,Write a letter to apologize to someone for som...,"Dear Alex,\n\nI want to sincerely apologize fo...",Prompt Level: 12-Upper Intermediate [SEP] Prom...
4,CALL0000291,5.0,5.0,5.0,5.0,100.000000,57ac94ab-2084-4a23-a6bf-61d71cd1900a,23c3d59a-61d5-470f-90d1-1217b0006929,10-Upper Intermediate,"An old friend has emailed you, describing what...",Subject: So Good to Hear From You! \n\nStan! ...,Prompt Level: 10-Upper Intermediate [SEP] Prom...


In [68]:
df = df[["text", "task_id", "level_title", "majority_value"]]
df = df.rename(columns={'majority_value': 'label'})
df.head()

,text,task_id,level_title,label
0,Prompt Level: 11-Upper Intermediate [SEP] Prom...,de7a1159-0c06-49bf-a273-7bdd195d6a32,11-Upper Intermediate,3.0
1,Prompt Level: 8-Intermediate [SEP] Prompt: You...,7ae47a96-722c-4b2b-ad07-3716f2aef85f,8-Intermediate,0.0
2,Prompt Level: 3-Beginner [SEP] Prompt: Write a...,cfb893f1-bee1-4894-99f3-43af284493a5,3-Beginner,5.0
3,Prompt Level: 12-Upper Intermediate [SEP] Prom...,03849587-d03b-45ed-8534-c34536867a8a,12-Upper Intermediate,5.0
4,Prompt Level: 10-Upper Intermediate [SEP] Prom...,23c3d59a-61d5-470f-90d1-1217b0006929,10-Upper Intermediate,5.0


In [69]:
# Apply the mapping to the 'labels' column
#df['label'] = df['label'].map(label_mapping)
df.dropna(subset=['label'], inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,text,task_id,level_title,label
0,Prompt Level: 11-Upper Intermediate [SEP] Prom...,de7a1159-0c06-49bf-a273-7bdd195d6a32,11-Upper Intermediate,3.0
1,Prompt Level: 8-Intermediate [SEP] Prompt: You...,7ae47a96-722c-4b2b-ad07-3716f2aef85f,8-Intermediate,0.0
2,Prompt Level: 3-Beginner [SEP] Prompt: Write a...,cfb893f1-bee1-4894-99f3-43af284493a5,3-Beginner,5.0
3,Prompt Level: 12-Upper Intermediate [SEP] Prom...,03849587-d03b-45ed-8534-c34536867a8a,12-Upper Intermediate,5.0
4,Prompt Level: 10-Upper Intermediate [SEP] Prom...,23c3d59a-61d5-470f-90d1-1217b0006929,10-Upper Intermediate,5.0


In [70]:
df["label"].value_counts()

label
5.0    930
2.0    556
4.0    466
3.0    351
0.0    340
1.0    118
Name: count, dtype: int64

In [71]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'task_id', 'level_title', 'label'],
    num_rows: 2761
})

In [72]:
from datasets import ClassLabel, Value, Sequence
new_features = ds.features.copy()
new_features["label"] = ClassLabel(names=[0, 1, 2, 3, 4])
ds = ds.cast(new_features)

# Step 1: Initial train/test split with stratification
train_test_ds = ds.train_test_split(test_size=0.20, seed=20)

# Step 2: Split the test set into half test, half validation
test_valid_split = train_test_ds['test'].train_test_split(test_size=0.5, seed=20)

# Step 3: Combine everything into a single DatasetDict
ds = DatasetDict({
    'train': train_test_ds['train'],
    'test': test_valid_split['train'],    # This becomes the test set
    'validation': test_valid_split['test']  # This becomes the validation set
})
ds

Casting the dataset:   0%|          | 0/2761 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 2208
    })
    test: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 276
    })
    validation: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 277
    })
})

In [73]:
# Verify label distribution
from collections import Counter

print("Train label counts:", Counter(ds['train']['label']))
print("Test label counts:", Counter(ds['test']['label']))
print("Validation label counts:", Counter(ds['validation']['label']))

Train label counts: Counter({5: 732, 2: 447, 4: 381, 3: 278, 0: 274, 1: 96})
Test label counts: Counter({5: 98, 2: 57, 4: 44, 3: 35, 0: 34, 1: 8})
Validation label counts: Counter({5: 100, 2: 52, 4: 41, 3: 38, 0: 32, 1: 14})


In [74]:
import json

# Fonction utilitaire pour sauvegarder un split en JSONL
def save_split_to_jsonl(dataset_split, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for record in dataset_split:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

# Sauvegarde des trois splits
save_split_to_jsonl(ds['train'], 'data/train.jsonl')
save_split_to_jsonl(ds['test'], 'data/test.jsonl')
save_split_to_jsonl(ds['validation'], 'data/validation.jsonl')

## Roberta SFT

In [75]:
import numpy as np 
import evaluate

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, cohen_kappa_score, classification_report

from scipy.stats import pearsonr

metric = evaluate.load("accuracy")

In [76]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convertir les logits en classes prédictes

    # 🎯 Exactitude (Accuracy)
    accuracy = accuracy_score(labels, predictions)

    # 🎯 Précision, Rappel et F1-score (pondérés)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    # 🎯 Score de Cohen's Kappa (pondéré)
    cohen_kappa = cohen_kappa_score(labels, predictions, weights="quadratic")

    # 🎯 Corrélation de Pearson
    pearson_corr, _ = pearsonr(labels, predictions)  # Retourne (coef, p-valeur), on garde seulement coef

     # 🎯 Classification Report
    class_report = classification_report(labels, predictions, output_dict=True)  # Get a dictionary of the report


    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "cohen_kappa": cohen_kappa,
        "pearson_corr": pearson_corr,
        "classification_report": class_report  # Add classification report to the return
    }


In [77]:
from datasets import load_dataset
from datasets import DatasetDict, Dataset

# Charger les fichiers JSONL en DatasetDict
dataset = DatasetDict({
    "train": load_dataset("json", data_files="data/train.jsonl")["train"],
    "test": load_dataset("json", data_files="data/test.jsonl")["train"],
    "valid": load_dataset("json", data_files="data/validation.jsonl")["train"]
})

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 2208
    })
    test: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 276
    })
    valid: Dataset({
        features: ['text', 'task_id', 'level_title', 'label'],
        num_rows: 277
    })
})


In [78]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

In [79]:
dataset["train"][0]

{'text': 'Prompt Level: 16-Upper Advanced [SEP] Prompt: Read an email from your friend and give advice on some stressful problems he is having. Type in the input box. Write 100-200 words. [SEP] Response: Hi Leon,\n\nI really sorry for hearing you are still having issues at work, I know you work can be kind of hectic and have some solutions, as you always ended up frazzled at the end of the day, why don you ruminate right after arriving home,  I find it soothing enough, and if you do it with yoga included it is totally worth it. Now talking about Tia, it is a shame things are going well, in this case if you want to, I can recommend you go to this beautiful lake called Calm Lake, which is surrounded by a tree, nature, it is a serene lake, you can take your time to assess yourself and make better decisions.\n\nWell Leon, I hope these suggestions help you a lot in order to cope those issues, keep me updated if you have any headway during the process. \n\nBest wishes,\n\nAnderson ',
 'task_

In [80]:
tok_test = tokenizer(dataset["train"][1]["text"], max_length=256, truncation=True)
tok_test

{'input_ids': [0, 35396, 3320, 12183, 35, 379, 12, 44694, 646, 3388, 510, 742, 42944, 35, 31043, 5, 2777, 47, 348, 2435, 11, 42, 8567, 7, 10, 92, 5674, 35, 2777, 2239, 4, 21062, 59, 743, 8, 2019, 14, 2777, 521, 747, 652, 4, 12738, 2512, 17769, 1142, 7, 244, 14192, 5, 2410, 4, 7773, 11, 5, 8135, 2233, 4, 21062, 3982, 12, 5714, 1617, 4, 646, 3388, 510, 742, 19121, 35, 11677, 11, 10, 92, 138, 16, 460, 3571, 6, 53, 67, 182, 17322, 4, 345, 32, 171, 383, 47, 240, 7, 1532, 1769, 4, 509, 9, 5, 78, 2019, 16, 7, 1346, 141, 5, 138, 1364, 358, 317, 33, 63, 308, 1492, 8, 2040, 4, 7411, 6, 5907, 3922, 47, 2563, 99, 7, 109, 6, 98, 47, 531, 465, 66, 1937, 4, 50118, 50118, 21518, 936, 16, 529, 92, 4025, 4, 370, 218, 17, 27, 90, 216, 106, 6, 8, 51, 218, 17, 27, 90, 216, 47, 4, 85, 64, 28, 1202, 7, 619, 233, 9, 5, 165, 11, 5, 1786, 4, 370, 33, 7, 28, 5192, 8, 3473, 1686, 7, 82, 6, 190, 114, 47, 32, 9152, 4, 50118, 50118, 22412, 6, 2239, 92, 8558, 50, 2257, 64, 185, 86, 4, 993, 451, 1057, 47, 7, 173, 1769

In [81]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

In [82]:
tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)
tokenized_valid = dataset["valid"].map(tokenize_function, batched=True)

Map:   0%|          | 0/2208 [00:00<?, ? examples/s]

Map:   0%|          | 0/276 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

In [83]:
unique_labels = set(dataset['train']['label'])
num_labels = len(unique_labels)
num_labels

6

In [84]:
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-large", num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
args = TrainingArguments(
    output_dir="../../../model_saved/roberta-large-ft-efcamdat-augmented",
    eval_strategy="steps",  # Évaluation aux mêmes intervalles que la sauvegarde
    save_strategy="steps",  # Sauvegarde tous les 500 steps
    save_steps=200,
    eval_steps=200,  # ⚠ IMPORTANT : Évaluation aux mêmes steps
    save_total_limit=4,  # Ne garde que 4 checkpoints max
    learning_rate=2e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="linear", 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    load_best_model_at_end=True,  
    metric_for_best_model="f1",
    logging_steps=100,
    fp16=True,
)

In [86]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [87]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss,Validation Loss


TrainOutput(global_step=828, training_loss=0.5889229829184675, metrics={'train_runtime': 299.1896, 'train_samples_per_second': 44.28, 'train_steps_per_second': 2.767, 'total_flos': 6173196781289472.0, 'train_loss': 0.5889229829184675, 'epoch': 6.0})

In [88]:
trainer.evaluate()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 1.1551573276519775,
 'eval_accuracy': 0.75,
 'eval_precision': 0.7535852468077532,
 'eval_recall': 0.75,
 'eval_f1': 0.7431084449411779,
 'eval_cohen_kappa': 0.9014164752905702,
 'eval_pearson_corr': 0.9070230789560548,
 'eval_classification_report': {'0': {'precision': 0.9411764705882353,
   'recall': 0.9411764705882353,
   'f1-score': 0.9411764705882353,
   'support': 34.0},
  '1': {'precision': 1.0,
   'recall': 0.5,
   'f1-score': 0.6666666666666666,
   'support': 8.0},
  '2': {'precision': 0.9111111111111111,
   'recall': 0.7192982456140351,
   'f1-score': 0.803921568627451,
   'support': 57.0},
  '3': {'precision': 0.5641025641025641,
   'recall': 0.6285714285714286,
   'f1-score': 0.5945945945945946,
   'support': 35.0},
  '4': {'precision': 0.4857142857142857,
   'recall': 0.38636363636363635,
   'f1-score': 0.43037974683544306,
   'support': 44.0},
  '5': {'precision': 0.7647058823529411,
   'recall': 0.9285714285714286,
   'f1-score': 0.8387096774193549,
   'sup

## Detailed Evaluation

In [90]:
list_topic = dataset["valid"]["task_id"]
list_t_set = set(list_topic)
unique_t = (list(list_t_set))

list_level = dataset["valid"]["level_title"]
list_l_set = set(list_level)
unique_l = (list(list_l_set))


In [94]:
list_r = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for t in unique_t:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['task_id'] == t)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "task_id": t,
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r.append(r)

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

In [95]:
list_r_level = []

# Assuming 'unique_t' is a list of unique item_ids and 'trainer' is already defined
for l in unique_l:  # Iterate over the first item in unique_t
    sub_ds = tokenized_valid.filter(lambda example: example['level_title'] == l)
    # Get predictions using the trainer
    predictions = trainer.predict(sub_ds)
    # Raw output logits (size [batch_size, num_classes])
    outputs = predictions.predictions
    # Convert logits to predicted class labels (taking the argmax across the classes)
    predicted_labels = np.argmax(outputs, axis=-1)
    ref_label = predictions.label_ids
    # Print or save the predicted classes (this will be a numpy array with the predicted class indices)
    ck = round(cohen_kappa_score(predicted_labels, ref_label, weights="quadratic"), 2)  
    pearson_corr, _ = pearsonr(ref_label, predicted_labels)
    accuracy = accuracy_score(ref_label, predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(ref_label, predicted_labels, average="weighted")

    r = {
        "level_title": sub_ds["level_title"][0],
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "ck": ck,
        "pearson": pearson_corr,
        "n_samples": len(sub_ds)
    }
    list_r_level.append(r)


Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

...

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

Filter:   0%|          | 0/277 [00:00<?, ? examples/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [96]:
import pandas as pd
df_eval_results = pd.DataFrame(list_r, columns=["task_id", "level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results.head(n=10)

,task_id,level_title,accuracy,precision,recall,f1,ck,pearson,n_samples
0,493dffa9-7513-4408-be4b-5b010a1bf051,10-Upper Intermediate,0.333333,0.407407,0.333333,0.355556,0.37,0.497863,9
1,dcdc664b-404c-407a-99a6-b9a1f99a859a,7-Intermediate,0.363636,0.545455,0.363636,0.430303,0.81,0.837415,11
2,9032f3ed-5da1-4efe-b4dd-c9470bb35d65,16-Upper Advanced,0.666667,0.633333,0.666667,0.617284,0.84,0.847094,9
3,de7a1159-0c06-49bf-a273-7bdd195d6a32,11-Upper Intermediate,0.666667,0.740741,0.666667,0.698413,0.88,0.881818,9
4,fa1e732b-7698-4b57-85da-907fc3660ec3,2-Beginner,0.333333,0.222222,0.333333,0.250000,0.86,0.965908,6
5,b4ba34ff-949d-4a82-9f56-ea472f177a71,13-Advanced,0.363636,0.200000,0.363636,0.256494,0.74,0.824022,11
6,43d154ea-bd72-4b54-9339-7c12f992b56a,15-Advanced,0.500000,0.479167,0.500000,0.469048,0.67,0.712525,8
7,23c3d59a-61d5-470f-90d1-1217b0006929,10-Upper Intermediate,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,8
8,8f1a0c16-d259-4737-af06-b6b6eb9e9799,13-Advanced,0.833333,0.833333,0.833333,0.833333,0.94,0.979796,6
9,2667a44c-a31b-4b3e-a33e-144d61884196,3-Beginner,0.375000,0.687500,0.375000,0.437500,0.84,0.926179,8


In [97]:
df_eval_results.to_csv("result_eval_data_roberta_large_writing_task_acc_sample.csv", index=False)

In [98]:
import pandas as pd
df_eval_results_level = pd.DataFrame(list_r_level, columns=["level_title", "accuracy", "precision", "recall", "f1", "ck", "pearson", "n_samples"])
df_eval_results_level.head(n=20)

,level_title,accuracy,precision,recall,f1,ck,pearson,n_samples
0,10-Upper Intermediate,0.647059,0.666667,0.647059,0.653119,0.63,0.679126,17
1,12-Upper Intermediate,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,19
2,16-Upper Advanced,0.842105,0.872180,0.842105,0.834308,0.89,0.887413,19
3,15-Advanced,0.777778,0.736953,0.777778,0.750265,0.84,0.854543,18
4,11-Upper Intermediate,0.650000,0.607143,0.650000,0.626923,0.88,0.903542,20
5,2-Beginner,0.647059,0.869748,0.647059,0.663000,0.77,0.794838,17
6,3-Beginner,0.625000,0.739583,0.625000,0.649242,0.95,0.957737,16
7,4-Elementary,0.833333,0.869444,0.833333,0.819400,0.75,0.732632,18
8,8-Intermediate,0.625000,0.504167,0.625000,0.554167,0.49,0.492524,16
9,14-Advanced,0.750000,0.900000,0.750000,0.786190,0.94,0.963761,20


In [99]:
df_eval_results_level.to_csv("result_eval_data_roberta_large_acc_sample_by_level.csv", index=False)